In [10]:
import pandas as pd

import mfnn

import warnings
warnings.filterwarnings("ignore")

#### Data import

In [2]:
df_ratings_in = pd.read_csv('data/ratings.csv')
df_ratings_in.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


#### Model

In [3]:
rec = mfnn.MFNNRecommender(
    data=df_ratings_in,
    user_field='userId',
    item_field='movieId',
    rating_field='rating',
)

In [4]:
rec.fit(dim=20, epochs=2)

In [5]:
rec.recommend(1, 10)

[4008, 7056, 177, 99085, 1324, 3757, 27441, 4895, 4875, 1981]

In [6]:
rec.save('noname')

In [7]:
rec_new = mfnn.MFNNRecommender(onload=True)

In [8]:
rec_new.load('noname')

In [9]:
rec_new.recommend(1, 10)

[4008, 7056, 177, 99085, 1324, 3757, 27441, 4895, 4875, 1981]